# Transportation Simulation using Agent Based approach 

### MESA Agenet Based library 

- all postcode are coming from  the dataset which has population and in-use postcodes
- This version is using Bing Map API
- In this version, Agent base simulation developed using MESA library and shows  faster results compare with my MESA free version


In [ ]:
! pip install DateTime
! pip install googlemaps
! pip install bs4
! pip install geopy
! pip install Mesa

In [ ]:
import pandas as pd
import random  #import randrange
import datetime as dt
#Googel API code
import googlemaps
import urllib
import numpy as np
import sys
from geopy import distance
import math
import geopy.distance
import time
import threading
from mesa.space import ContinuousSpace
from mesa import Agent, Model
from mesa.time import SimultaneousActivation


#### feature dumper function:

1-  Traffic Features: 
        - Segment points
        - Time
        - Volume of traffic
        - Speed of traffic
        - Road type
        - Number of lane
        - Speed limit
        - Number of junctions
        - Road length
        - Head count/households count in 5 miles of road segments = sarshomar
        - Number of active postcode near the road
        - Lat/lang of start and end of this segments
        - How near is to business district

2-  Population and job demographic shuffle  function
        - Add weighted factor to postcodes for job and residential area, using this ref: 
        - ref: http://www.plumplot.co.uk/





In [ ]:
Df_everySEGs=pd.DataFrame(columns=['segmentPoints'])
Df_SEGsfull=pd.DataFrame(columns=['time','segmentPoints','roadType','VolumeTrafic','Color_Weight'])

gKEY = ''# Google map API key 
KEY = "" # Bing map API key 
folder=''# inset your databse directory 

dataset = pd.read_csv('postcodesNew.csv')

### Importing  Postcode dataset for places 
Location for gym and shoping centre manually added manually for busineses in the city; beacuse  Bing Local search is available now for USA only.


In [ ]:
dataset_gym =pd.read_csv('GYMPostcodecity.csv')
dataset_shop=pd.read_csv('ShopingCentrecity.csv')
dataset_school=pd.read_csv('Schoolcity.csv')


### Importing  Postcode dataset for city 

In [ ]:
#dataset=  dataset.dropna(subset=['Population'], inplace=True)
dataset = dataset[dataset['Households']>=5]
dataset=dataset.reset_index(drop=True)

postcode=list(dataset['Postcode'])
#demografic= pd.read_csv("filename.csv")
sim_date=[2019, 12, 15] #year,month,day 


In [ ]:
####    Usefull functions:
def random_time(start,h_shift,min_var):#H_shift is the hours after zero_time that one's day starts, min_var is minut variation from the nominal start day time, start is date time with this format (2018, 12, 20,8,00)
        date_time = start + dt.timedelta(hours=h_shift,minutes=random.randrange(min_var),seconds=0) #timedelta(weeks=40, days=84, hours=23,minutes=50, seconds=600)  # adds up to 365 days       
        #print (date_time.strftime("%d/%m/%y %H:%M"))
        #print (date_time.strftime("%H:%M:%S"))
        return date_time
    

import urllib, json

def LatLng(where):

    MyUrl = ('http://dev.virtualearth.net/REST/v1/Locations/UK/'
             +where.replace(' ', '')+
            '?&maxResults=1&key=%s') % (KEY)
    response = urllib.request.urlopen(MyUrl)
    jsonRaw = response.read()
    jsonData = json.loads(jsonRaw.decode())
    return jsonData['resourceSets'][0]['resources'][0]['geocodePoints'][0]['coordinates']

dataset_gym['latlng']=dataset_gym.loc[:,'Postcode']
dataset_shop['latlng']=dataset_shop.loc[:,'Postcode']



for i in range(len(dataset_gym)): 
    try: 
        dataset_gym.loc[i,'latlng']=LatLng(dataset_gym.loc[i,'Postcode'])
    except:
        dataset_gym.drop(i)
for i in range(len(dataset_shop)): 
    try:
        dataset_shop.loc[i,'latlng']=LatLng(dataset_shop.loc[i,'Postcode']) 
    except:
        dataset_shop.drop(i)
        
def places_finder(where,radius,types): # WORKS ONLY FOR US, not for UK yet
    #making the url
    latlng=LatLng(where)
    
    LOCATION = str(latlng[0]) + "," + str(latlng[0])+str(radius)

    MyUrl = ('https://dev.virtualearth.net/REST/v1/LocalSearch/'
            '?query=%s'
            '&userLocation=%s'
            '&key=%s') % (types,LOCATION, KEY)
    #grabbing the JSON result
    response = urllib.request.urlopen(MyUrl)
    jsonRaw = response.read()
    jsonData = json.loads(jsonRaw.decode())
    location=jsonData['results'][0]['geometry']['location']
    return  str(location['lat'])+','+str(location['lng'])
  
def places_picker(where,radius,from_list): # picks one location within the designated radius from_list
    try:
        where=LatLng(where)
        ind=0
        dist=20000000 # just a big number
        for i in list(from_list['latlng']):
            if radius=='nearest' and dist>geopy.distance.distance(tuple(i), tuple(where)).m :
                  dist=geopy.distance.distance(tuple(i), tuple(where)).m
                  place=from_list.loc[ind,'Postcode']
     
                
            elif radius!='nearest':
                if geopy.distance.distance(tuple(i), tuple(where)).m<radius:
                    place=from_list.loc[ind,'Postcode']#tuple(i)
                    break
            else:pass
            ind+=1
    except: place=random.choice(list(from_list.loc[:,'Postcode'])) 
    
    return place       


def init_list_of_objects(size,item=''):
    list_of_objects = [list(item)]
    for i in range(0,size):
        list_of_objects.append( list(item) ) #different object reference each time
    return list_of_objects


### define trips rules

In [ ]:
def trips(life_style,home_adrs,work_adrs,gym_adrs,shop_adrs,school_adrs):
        zero_time = dt.datetime(sim_date[0],sim_date[1],sim_date[2],00,00)   

        trip_nodes={'non_parent':[
                        ['HOME','WORK','HOME'],
                        ['HOME','WORK','GYM','HOME'],
                        ['HOME','WORK','SHOP','HOME'],
                        ['HOME','WORK','GYM','SHOP','HOME'],
                        ['HOME','WORK','SHOP','GYM','HOME'],
                        ['HOME','GYM','WORK','HOME'],
                        ['HOME','GYM','WORK','SHOP','HOME']], 
                    'parent':[
                        ['HOME','SCHOOL','HOME','SCHOOL','HOME'],
                        ['HOME','SCHOOL','WORK','SCHOOL','HOME'],
                        ['HOME','SCHOOL','WORK','GYM','HOME'],
                        ['HOME','WORK','SCHOOL','SHOP','HOME'],
                        ['HOME','WORK','GYM','SHOP','HOME'],
                        ['HOME','WORK','SHOP','GYM','HOME'],
                        ['HOME','GYM','WORK','HOME'],
                        ['HOME','GYM','WORK','SHOP','HOME']]   
                    }   
        trip_nodes={'parent':[['HOME','WORK','HOME']],'non_parent':[['HOME','WORK','HOME']]}

        steps=random.choice(trip_nodes[life_style])
        trips=[]
        for i in range(len(steps)-1):
            trips.append(steps[0+i:2+i])
            # before work activities
            if trips[i]==['HOME','GYM'] : trips[i].append('leave');trips[i].append(random_time(zero_time,6,40))
            elif trips[i][1]=='SCHOOL' :trips[i].append('arrive');trips[i].append(random_time(zero_time,8,15))
            elif trips[i][0]=='SCHOOL' and trips[i][1]=='WORK' :trips[i].append('arrive');trips[i].append(random_time(trips[-2][-1],1,5))
            elif trips[i][1]=='WORK' :trips[i].append('arrive');trips[i].append(random_time(zero_time,8,45))
            # after work activities : they are timed based on last activity time (trips[-1][-1])
            
            elif trips[i][0]=='WORK' :trips[i].append('leave');trips[i].append(random_time(trips[-2][-1],8,30)) #time spend at work: 8h
            elif trips[i][0]=='GYM'  :trips[i].append('leave');trips[i].append(random_time(trips[-2][-1],1,30))#time spend at gym: 1h
            elif trips[i][0]=='SHOP' :trips[i].append('leave');trips[i].append(random_time(trips[-2][-1],0,50))
            elif trips[i][0]=='SCHOOL' :trips[i].append('leave');trips[i].append(random_time(trips[-2][-1],0,10))
            else : pass
        
        #assigning actual addresses to the trip lists:
        actual_places = {'HOME':home_adrs, 'WORK':work_adrs, 'GYM':gym_adrs, 'SHOP':shop_adrs,'SCHOOL':school_adrs}
        for i in range(len(trips)):
            trips[i]=[actual_places.get(n, n) for n in trips[i]]
        #print (trips)
        return trips  
        
          


          


### Bing Direction

In [ ]:
def Bing_direction(start_point , end_point ,timetype , datetime , mode="Driving"):
    #making the url
    travelMode=mode
    start_point=urllib.parse.quote(start_point, safe='')
    end_point=urllib.parse.quote(end_point, safe='')
    datetime=urllib.parse.quote(str(datetime), safe='')    
    MyUrl = ('http://dev.virtualearth.net/REST/V1/Routes/'
            +travelMode+
            '?wp.0=%s'
            '&wp.1=%s'
            '&timeType=%s'
            '&dateTime=%s'
            '&routePathOutput=Points' #delete
            '&tolerances=0.05'   #delete
            '&key=%s') % (start_point, end_point, timetype,datetime, KEY) #timetype,datetime,
    #grabbing the JSON result
    #print (MyUrl)
    response = urllib.request.urlopen(MyUrl)
    jsonRaw = response.read()
    jsonData = json.loads(jsonRaw.decode())
    trip_data=jsonData["resourceSets"][0]["resources"][0]#["routeLegs"][0]['itineraryItems']
    return  trip_data



### find nearest

In [ ]:
#["resourceSets"][0]["resources"][0]["routeLegs"][0]['itineraryItems']

def find_nearest_index(array, value):
    idx = np.array([np.linalg.norm(x+y) for (x,y) in np.array(array)-np.array(value)]).argmin()
    return idx

def add_segmentPoints(steps,points):

    for i in range(len(steps)-1):
        
        try:frm=points.index(steps[i]['maneuverPoint']['coordinates'])
        except : frm=find_nearest_index(points, steps[i]['maneuverPoint']['coordinates'])
            
        try:to=points.index(steps[i+1]['maneuverPoint']['coordinates'])+1
        except: to=find_nearest_index(points, steps[i+1]['maneuverPoint']['coordinates'])+1
        
        p=points[frm:to]
        steps[i]['segmentPoints']=p
        #distance calculations between each two segment points 
        dists=[0]
        for j in range(len(p)-1):
            dists.append(geopy.distance.distance(p[j],p[j+1]).m) #returns the distance in 'm' , replace it with 'km' if needed
        steps[i]['segmentPoints_distance']=dists            
                            


### route_generator rules

In [ ]:
def route(trip_legs): # BING
        routes=[]
        #print (type(trip_legs))
        for leg in trip_legs:  
            #leg[0],leg[1] =origin and destination of each leg,leg[2]is departure or arrival, leg[-1] is the time
            if leg[2]=='leave':
                directions_result = Bing_direction(leg[0],leg[1],'Departure',leg[-1],mode="Driving")
            else: directions_result = Bing_direction(leg[0],leg[1],'Arrival',leg[-1],mode="Driving")                                 
            #extracting the key infor ployline points from a direcion_results 
            route_info=directions_result 
            #key info : start time, start location, total trip disttance, total trip time ,end location, end time
            if leg[2]=='leave':
                start_time=leg[-1];
                end_time=leg[-1] + dt.timedelta(seconds=route_info['routeLegs'][0]['travelDuration'])#this calculates the end time if we know the leave time

            else: 
                start_time=leg[-1] - dt.timedelta(seconds=route_info['routeLegs'][0]['travelDuration']) #this calculates the start time if we know the arrival time
                end_time=leg[-1]
        
            leg_key_info={'start_time':start_time,
                                'start_location':{'lat':route_info['routeLegs'][0]['actualStart']['coordinates'][0],'lng':route_info['routeLegs'][0]['actualStart']['coordinates'][1]},
                                'distance':route_info['travelDistance']*1000,
                                'duration':route_info['travelDuration'],
                                'end_location':{'lat':route_info['routeLegs'][0]['actualEnd']['coordinates'][0],'lng':route_info['routeLegs'][0]['actualEnd']['coordinates'][1]},
                                'end_time':end_time
                            }   
            # adds the segment points with the list of distances to each step of the route info
            add_segmentPoints(route_info["routeLegs"][0]['itineraryItems'],route_info["routePath"]['line']['coordinates'])
            #extracting the ployline points from a route                                                                                                         
            routes.append({'leg_key_info':leg_key_info,'steps':route_info["routeLegs"][0]['itineraryItems']})
        #the route is a list of trip route/info for each leg, for instance [0] returns 1st leg and will containe below dictionaries: 'leg_key_info' and 'steps'
        # for l in routes:
        #     print ('start->: ', l['leg_key_info']['start_location'], '@',l['leg_key_info']['start_time'])
        #     print ('--->end: ', l['leg_key_info']['end_location'], '@',l['leg_key_info']['end_time'])
        #     print('---------------------------------------------------------------------------------')
        return routes 




In [ ]:
class Spinner:
    busy = False
    delay = 0.1
    @staticmethod
    def spinning_cursor():
        while 1: 
            for cursor in '|/-\\': yield cursor
    def __init__(self, delay=None):
        self.spinner_generator = self.spinning_cursor()
        if delay and float(delay): self.delay = delay
    def spinner_task(self):
        while self.busy:
            sys.stdout.write('\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b'+next(self.spinner_generator)+' WORKING...')
            sys.stdout.flush()
            time.sleep(self.delay)
    def start(self):
        self.busy = True
        threading.Thread(target=self.spinner_task).start()
    def stop(self):
        self.busy = False
        time.sleep(self.delay)        
        sys.stdout.write('\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b'+'FINISHED!')


In [ ]:
### City dimensions

In [ ]:
x_min,x_max=-8.7,1.9
y_min,y_max=47.0,62.0

In [ ]:
class CityModel(Model):
    """A model with some number of agents."""
    def __init__(self, N):
        
        self.num_agents = N
        self.AreaMap = ContinuousSpace(x_max, y_max, True, x_min, y_min)
        self.schedule = SimultaneousActivation(self)
        # Create agents
        life_style_distribution=np.round(np.array([300000,100000])*N/400000.0)# total number of non_parent with cars in this city , total parent with cars in this city
        counter=0
        for i in range(self.num_agents):
            
            h=random.choice(postcode)#sample_home_postcode);
            w=random.choice(postcode)#sample_work_postcode)
        
            gym=places_picker(h,'nearest',dataset_gym)
            shop=places_picker(h,'nearest',dataset_shop)
            school=places_picker(h,'nearest',dataset_school)
            
            
            if counter<= life_style_distribution[0]:
                counter+=1
                a = CityAgent(i, h,'gender',32,'non_parent',1,w,gym,shop,school,self)
            elif life_style_distribution[0]<counter<= life_style_distribution[1]+life_style_distribution[0]:
                counter+=1
                a = CityAgent(i, h,'gender',32,'parent',3,w,gym,shop,school,self)
            else: pass
            
            self.schedule.add(a)
            xy=LatLng(h)
            x = xy[0]
            y = xy[1]
            self.AreaMap.place_agent(a, (x, y))
#            sys.stdout.write('\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b'+'Agent: '+str(1+i)+' / '+str(N))
#            sys.stdout.flush()
            print('Agent: '+str(1+i)+' / '+str(N))
    def step(self):
        try: self.schedule.step()
        except: pass
        



In [ ]:
#### #  Add weather impact here on the road speed : not complete yet


In [ ]:
class CityAgent(Agent):
    """ An agent with fixed initial location."""
    def __init__(self, unique_id, home_adrs,gender,age,life_style,house_hdcount,work_adrs,gym_adrs,shop_adrs,school_adrs,model):
        super().__init__(unique_id, model)
        self.id=unique_id
        self.home_adrs=home_adrs
        self.work_adrs=work_adrs
        self.gym_adrs=gym_adrs
        self.shop_adrs=gym_adrs
        self.school_adrs=school_adrs

        self.gender=gender
        self.age=age
        self.life_style=life_style
        self.house_hdcount=house_hdcount
   
        # additional places:
#        self.gym_adrs= places_picker(self.home_adrs,5000,dataset_gym)
#        self.nursery_adrs=(51.764156, -1.258917)# places_finder(self.home_adrs,5000,'nursery')
#        self.shop_adrs= places_picker(self.home_adrs,5000,dataset_shop)
        # Agent's trips and associated routes:
        self.trips=trips(self.life_style,self.home_adrs,self.work_adrs,self.gym_adrs,self.shop_adrs,self.school_adrs)
        try:self.route=route(self.trips)
        except: print(self.trips)

       
    def loc8(self,t): #location of agent at time = t (datetime.datetime type)
        R=self.route[:-1]

        r=[]
        for i in list(R[0]['steps'][:-1]):
            r.append(i['segmentPoints'])
            
        for l in R:
            
            if  t <= l['leg_key_info']['start_time']: 
                loc=(l['leg_key_info']['start_location']['lat'],l['leg_key_info']['start_location']['lng'])
                return loc, None , None, None #(loc[1],loc[0])
                break
                

            elif l['leg_key_info']['start_time'] < t <= l['leg_key_info']['end_time']:
                t=t-l['leg_key_info']['start_time']
                t=int(t.total_seconds())
                i=0 #this is the counter from one step to another
                while t>l['steps'][i]['travelDuration']:
                    t=t-l['steps'][i]['travelDuration']
                    if i==len(l['steps'])-2 : break
                    i+=1 #this 'i' once out of the shows in which step of the leg the agent is 
                #making a displacement vector from start position (ps) and end position (pe)

                step_points=l['steps'][i]['segmentPoints']
                step_length=l['steps'][i]['segmentPoints_distance']
                step_velocity=1000*l['steps'][i]['travelDistance']/l['steps'][i]['travelDuration']
                step_time=np.array(step_length)/step_velocity # this list 'step_time' starts with '0' as first item

                ii=0 #this is the counter within one step from one point to another
                while ii+1<len(step_time) and t>step_time[ii+1]:
                    t=t-step_time[ii+1]
                    ii+=1
                try: pe=step_points[ii+1];ps=step_points[ii];tt=t/step_time[ii+1]
                except:pe=step_points[-1];ps=step_points[-2];tt=t/step_time[-1]
                
                vec=np.array(pe)-np.array(ps)
                #making a partial displacement vector based on the time passed from start position
                vec=tt*vec
                #current position : ps + partial vector
                loc=tuple(ps+vec)# array
                del r[i]
                return loc ,l['steps'][i]['segmentPoints'], l['steps'][i]['details'][0]['roadType'],r
                break
            
            else: 
                loc=(l['leg_key_info']['end_location']['lat'],l['leg_key_info']['end_location']['lng']);
                return loc , None , None, None
  
    def move(self):

        new_position,segment_points, road_type,other_segs = self.loc8(t)
        #print ('time:',t,'  new',new_position)
        #print ('other_segs:',other_segs)
        self.model.AreaMap.move_agent(self, new_position)
        LOCs[time_list.index(t)].append(new_position)
        global Df_everySEGs
        if other_segs!=None:
            for i in range(len(other_segs)):
                Df_everySEGs= Df_everySEGs.append({'segmentPoints':other_segs[i]},ignore_index=True)
        
        global Df_SEGsfull
        if segment_points!=None:
            Df_SEGsfull=Df_SEGsfull.append({'time':t,'segmentPoints':segment_points,'roadType':road_type,'VolumeTrafic':1,'Color_Weight':1},ignore_index=True)
#            try:
#                Df_SEGs.loc[(Df_SEGs['time']==t),'VolumeTrafic']+=1#(Df_SEGs['segmentPoints']==segment_points) & 
#
#            except:            
#                Df_SEGs=Df_SEGs.append({'time':t,'segmentPoints':segment_points,'roadType':road_type,'VolumeTrafic':1,'Color_Weight':1},ignore_index=True)
#            
#        print(self.id)
#        sys.stdout.write('\b\b\b\b\b\b\b\b\b\b\b\b'+str(self.id))
#        sys.stdout.flush()


        
    def step(self):
        self.move()
        #self.Volume_calc()
    def advance(self):
        pass



#### Simulation